# Cvičenie 3: Bellmanove rovnice

Ako už viete z prednášky, v učení posilňovaním sú úlohy resp. prostredia často reprezentované ako Markovove rozhodovacie procesy. Na dnešnom cvičení sa pozrieme na to, ako vieme vyriešiť problém, ktorý predstavuje takéto prostredie na príklade dvojdimenzionálneho sveta gridworld, v ktorom sa agent snaží dostať do cieľa a vyhýbať sa pasci. Cieľom je teda nájsť optimálnu politiku na pohybovanie sa v gridworlde - najkratšiu trasu. Využijeme pritom Bellmanove rovnice očakávania a funkciu optimality.

## Gridworld

Výpočty budeme aplikovať pre konkrétny gridworld veľkosti *3x3* s jedným cieľom a jednou pascou. Cieľová pozícia a pasca sú terminálne stavy, takže vieme, že ich očakávaná hodnota je 0. Agent má k dispozícii štyri akcie na každej pozícii: pohyb na sever, východ, juh a západ. Dolná a pravá časť sveta (biele pozície) je úplne deterministická, pričom na pozíciach uznačených svetlomodrou farbou fúka silný vietor, ktorý môže agenta posunúť na východ aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na východ je $0.4$. Ak sa agent pohybuje na západ, s touto pravdepodobnosťou ostane na svojej pôvodnej pozícii (vietor a jeho pohyb sa rušia). Ak vyberie pohyb na východ, určite sa tam dostane.

<img src="lab03/gridworld_map.jpg" width="300">

Pre lepšiu predstavu prechodov daný svet vieme reprezentovať formálne ako MDP. Pri prechodoch sú označené akcie, pravdepodobnosť prechodu a odmena pre vykonanie danej akcie.

<img src="lab03/gridworld_mdp.jpg" width="600">

## Bellmanova rovnica očakávania

V prvom kroku vypočítame očakávanú hodnotu jednotlivých stavov pri náhodnej politike. Použijeme na to Bellmanovu rovnicu očakávania:

$v_{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a|s) \ (\sum_{r \in \mathcal{R}} r \sum_{s' \in \mathcal{S}} p(s', r|s, a) + \gamma \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r|s, a) \  v_{\pi}(s'))$

Keďže politika je náhodná, $\pi(a|s) = 0.25$ pre všetky stavy. Diskontný faktor $\gamma = 0.8$. Pre odmeny budeme používať nasledovné označenia:

$r_{g} = 10$

$r_{t} = -10$

$r_{s} = -1$

$r_{i} = 5$

Napríklad pre stav $s_{11}$ vieme vypočítať hodnotu nasledovne:

$v_{\pi}(s_{11}) = \pi(N|s_{11}) \cdot (r_{s} \cdot p(s_{11}, r_{s}| s_{11}, N) + \gamma \cdot p(s_{11}, r_{s}|s_{11}, N) \cdot v_{\pi}(s_{11})) + \\ + \pi(E|s_{11}) \cdot (r_{s} \cdot p(s_{12}, r_{s}| s_{11}, E) + \gamma \cdot p(s_{12}, r_{s}|s_{11}, E) \cdot v_{\pi}(s_{12})) + \\ + \pi(S|s_{11}) \cdot (r_{s} \cdot p(s_{21}, r_{s}| s_{11}, S) + \gamma \cdot p(s_{21}, r_{s}|s_{11}, S) \cdot v_{\pi}(s_{21})) + \\ + \pi(W|s_{11}) \cdot (r_{s} \cdot p(s_{11}, r_{s} | s_{11}, W) + \gamma \cdot p(s_{11}, r_{s}|s_{11}, W) \cdot v_{\pi}(s_{11}))$

$v_{\pi}(s_{11}) = 0.25 \cdot (-1.0 \cdot 1.0 + 0.8 \cdot 1.0 \cdot v_{\pi}(s_{11})) + \\ + 0.25 \cdot (-1.0 \cdot 1.0 + 0.8 \cdot 1.0 \cdot v_{\pi}(s_{12})) + \\ + 0.25 \cdot (-1.0 \cdot 1.0 + 0.8 \cdot 1.0 \cdot v_{\pi}(s_{21})) + \\ + 0.25 \cdot (-1.0 \cdot 1.0 + 0.8 \cdot 1.0 \cdot v_{\pi}(s_{11}))$

$v_{\pi}(s_{11}) = 0.25 \cdot (-1 + 0.8 \cdot v_{\pi}(s_{11})) + 0.25 \cdot (-1 + 0.8 \cdot v_{\pi}(s_{12})) + 0.25 \cdot (-1 + 0.8 \cdot v_{\pi}(s_{21})) + 0.25 \cdot (-1 + 0.8 \cdot v_{\pi}(s_{11}))$

$v_{\pi}(s_{11}) = -1 + 0.4 \cdot v_{\pi}(s_{11}) + 0.2 \cdot v_{\pi}(s_{12}) + 0.2 \cdot v_{\pi}(s_{21})$

Keď to upravíme na formu lineárnych rovníc:

$0.6 \cdot v_{\pi}(s_{11}) - 0.2 \cdot v_{\pi}(s_{12}) - 0.2 \cdot v_{\pi}(s_{21}) = -1$

**Úloha:** Zostavte rovnice pre ostatné stavy (okrem $s_{13}$ a $s_{22}$).

Pre všetky stavy dostaneme spolu sedem lineárnych rovníc o sedem neznámych. Následne môžeme vyriešiť túto sústavu a tak dostaneme očakávané hodnoty $v_{\pi}(s)$ pri politike $\pi$.

**Úloha:** Vyriešte sústavu zostavených lineárnych rovníc v Pythone. Použite pritom funkciu `numpy.linalg.solve`.

In [ ]:
import numpy as np

A = np.array([
    [],
    [],
    [],
    [],
    [],
    [],
    []
])
B = np.array([
    
])
X = np.linalg.solve(A, B)

## Bellmanova funkcia optimality

Bellmanove rovnice môžeme využiť aj pri hľadaní optimálnej politiky pre daný svet. Zameriame sa pritom na očakávanú hodnotu dvojíc stav-akcia, teda chceme zistiť, ako veľmi je daná akcia vhodná pre daný stav. Hodnotu určíme podľa vzorca:

$q(s, a) = \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r|s, a) \cdot (r + \gamma \cdot v(s'))$

Akcia je optimálna ak maximalizuje odmeny počas interakcie:


$q_{*}(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a) \cdot v_{*}(s')$

$q_{*}(s, a) = \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r|s, a) (r + \gamma \cdot \underset{a}{max} q_{*}(s', a'))$

Pre náš príklad teda vieme napísať rovnice:

$q(s_{11}, N) = p(s_{11}, r_{s}|s_{11}, N) \cdot (r_{s} + \gamma \cdot v_{*}(s_{11}))$

$q(s_{11}, N) = 1.0 \cdot (-1 + 0.8 \cdot v_{*}(s_{11}))$

$q(s_{11}, N) = -1 + 0.8 \cdot v_{*}(s_{11})$

$q(s_{11}, N) - 0.8 \cdot v_{*}(s_{11}) = -1$

A obdobne pre ďalšie akcie:

$q(s_{11}, E) - 0.8 \cdot v_{*}(s_{12}) = -1$

$q(s_{11}, S) - 0.8 \cdot v_{*}(s_{21}) = -1$

$q(s_{11}, W) - 0.8 \cdot v_{*}(s_{11}) = -1$

**Úloha:** Napíšte rovnice pre všetky zvyšné kombinácie stav-akcia (okrem stavov $s_{13}$ a $s_{22}$). Vychádzame zo znalosti, že $v_{*}(s_{13}) = 0$ a $v_{*}(s_{22}) = 0$.

Dostali sme 28 rovníc (7 stavov $\times$ 4 akcie), máme ale spolu 35 neznámych (28 hodnôt stav-akcia + 7 maximálnych hodnôt stavov). Našu sústavu môžeme rozšíriť o predpoklady, že maximálna hodnota stavu musí byť najväčšia z hodnôt stav-akcia pre daný stav, napr.:

$v_{*}(s_{11}) = max(q(s_{11}, N), q(s_{11}, E), q(s_{11}, S), q(s_{11}, W))$

To ale ešte stále nevieme jednoducho napísať ako rovnicu, práve preto hodnoty vypočítame pomocou SMT solvera. Ak naše rovnice a predpoklady zapíšeme napríklad [do SMT solvera Z3](https://compsys-tools.ens-lyon.fr/z3/index.php), [naše riešenie môže vyzerať takto.](lab03/optimal_smt.smt)

Alternatívne riešenie je vyskúšať všetky možné politiky a zistiť, či sú optimálne. Takto naše predpoklady na $v_{*}(s)$ vieme zapísať ako rovnicu, napríklad ak akcia $E$ je optimálna zo stavu $s_{11}$: $v_{*}(s_{11}) - q(s_{11}, E) = 0$. Následne vyriešime sústavu lineárnych rovníc a skontrolujeme si, či hodnota $v_{*}(s)$ je naozaj najväčšia z hodnôt $q(s, a) \forall a \in \mathcal{A}$. Takto nájdeme všetky optimálne politiky. [Ukážkové riešenie v Pythone nájdete tu.](lab03/bellman_optimal.py)